In [1]:
from selenium import webdriver
import time 
from bs4 import BeautifulSoup
import sqlalchemy
import urllib
import shutil
import pandas as pd

In [2]:
engine = sqlalchemy.create_engine('mssql+pyodbc://' + 'LAPTOP-HAPH02BA' + '/' +'NBA'+ "?" + 'driver=SQL Server Native Client 11.0')

In [3]:
con = engine.connect()

In [4]:
listado_jugadores = 'select distinct(Player) from mvp where year = 2022'

In [5]:
df_jugadores = pd.DataFrame(con.execute(listado_jugadores).fetchall())

In [6]:
df_jugadores.columns = ['Player']

In [7]:
lista_jugadores = list(df_jugadores['Player'])

In [8]:
for jugador in lista_jugadores:
    driver = webdriver.Chrome()
    url = "https://www.basketball-reference.com/"
    driver.get(url)
    time.sleep(3)
    Element = driver.find_element_by_css_selector('#header > div.search > form > div > div > input.ac-input.completely')
    Element.send_keys(jugador)
    Element = driver.find_element_by_css_selector('#header > div.search > form > input[type=submit]:nth-child(2)')
    Element.click()
    try:
        html = driver.page_source
        soup = BeautifulSoup(html)
        link_imagen = soup.findAll('img',attrs={'itemscope':'image'})[0]['src']
        url_imagen = link_imagen
        urllib.request.urlretrieve(url_imagen, "Foto de Jugadores/"+jugador +".jpg")
    except:
            try:
                Element = driver.find_element_by_css_selector('#players > div.search-item > div.search-item-name > a')
                Element.click()
                html = driver.page_source
                soup = BeautifulSoup(html)
                link_imagen = soup.findAll('img',attrs={'itemscope':'image'})[0]['src']
                url_imagen = link_imagen
                urllib.request.urlretrieve(url_imagen, "Foto de Jugadores/"+jugador +".jpg")
            except:
                
                try:
                    Element = driver.find_element_by_css_selector('#players > div.search-item > div.search-item-name > strong > a')
                    Element.click()
                    html = driver.page_source
                    soup = BeautifulSoup(html)
                    link_imagen = soup.findAll('img',attrs={'itemscope':'image'})[0]['src']
                    url_imagen = link_imagen
                    urllib.request.urlretrieve(url_imagen, "Foto de Jugadores/"+jugador +".jpg")
                except:                 
                    ruta_copiar = 'no pic.jpg'
                    ruta_pegar = 'Foto de Jugadores/' + jugador + '.jpg'
                    shutil.copy(ruta_copiar, ruta_pegar)
                    driver.close()